In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
import joblib
import gc

# =========================================
# 1. CẤU HÌNH & ĐỌC DỮ LIỆU
# =========================================
INPUT_FILE = "../data_clear2/train_split.csv"
OUTPUT_DIR_STEP1 = "processed_data"
OUTPUT_DIR_FINAL = "processed_data_final_v1"
os.makedirs(OUTPUT_DIR_STEP1, exist_ok=True)
os.makedirs(OUTPUT_DIR_FINAL, exist_ok=True)

print(f"⏳ Đang đọc dữ liệu từ {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE)
print(f"✅ Đã đọc dữ liệu. Shape: {df.shape}")

# =========================================
# 2. XỬ LÝ SƠ BỘ (Label Encoding & NaN)
# =========================================
print("\n🔄 [BƯỚC 1] Xử lý sơ bộ (Encoding & NaN)...")
df_processed = df.copy()

# 2.1. Label Encoding cho các biến phân loại
categorical_cols = df_processed.select_dtypes(include=['object']).columns
print(f" -> Tìm thấy {len(categorical_cols)} cột categorical cần mã hóa.")
label_encoders = {}
for col in categorical_cols:
    df_processed[col] = df_processed[col].astype(str).fillna('unknown')
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le
print(" -> Hoàn tất Label Encoding.")

# 2.2. Tách nhãn và định danh để xử lý đặc trưng an toàn
y_label = df_processed['isFraud']
df_features = df_processed.drop(columns=['isFraud', 'TransactionID'])
transaction_ids = df_processed['TransactionID'] # Giữ lại TransactionID
transaction_dts = df_processed['TransactionDT'] if 'TransactionDT' in df_processed.columns else None

# =========================================
# 3. LỌC ĐẶC TRƯNG (Feature Selection)
# =========================================
print("\n🔍 [BƯỚC 2] Lọc đặc trưng (Feature Selection)...")

# 3.1. Loại bỏ đặc trưng ít biến thiên (Variance Threshold)
print(" -> Đang lọc đặc trưng ít biến thiên (Variance < 0.05)...")
constant_filter = VarianceThreshold(threshold=0.05)
constant_filter.fit(df_features)
features_to_keep_var = df_features.columns[constant_filter.get_support()]
df_features = df_features[features_to_keep_var]
print(f"    => Còn lại {df_features.shape[1]} đặc trưng.")

# 3.2. Loại bỏ đặc trưng tương quan cao (Correlation)
print(" -> Đang lọc đặc trưng tương quan cao (> 0.99)...")
corr_matrix = df_features.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.99)]
df_features = df_features.drop(columns=to_drop_corr)
print(f"    => Đã loại bỏ {len(to_drop_corr)} đặc trưng dư thừa. Còn lại {df_features.shape[1]} đặc trưng.")

# 3.3. Chọn đặc trưng quan trọng bằng RandomForest
print(" -> Đang chạy RandomForest để xếp hạng độ quan trọng...")
rf_model = RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42) # Giảm estimators để chạy nhanh hơn
rf_model.fit(df_features, y_label)
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'feature': df_features.columns, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Chọn Top 40 đặc trưng tốt nhất
TOP_N = 100
top_n_features = feature_importance_df['feature'].head(TOP_N).tolist()
print(f"    => Đã chọn ra {TOP_N} đặc trưng quan trọng nhất.")

# Lưu Metadata
feature_importance_df.to_csv(os.path.join(OUTPUT_DIR_STEP1, "feature_importance_ranking.csv"), index=False)
joblib.dump(top_n_features, os.path.join(OUTPUT_DIR_STEP1, "features_names.joblib"))

# =========================================
# 4. TẠO DATASET TINH GỌN & CHUẨN HÓA
# =========================================
print("\n⚖️ [BƯỚC 3] Tạo dataset cuối cùng và Chuẩn hóa (Scaling)...")


# 4.1. Tạo DataFrame chỉ chứa Top features + các cột cần thiết
essential_cols = ['TransactionID', 'isFraud']
if 'TransactionDT' in df.columns: essential_cols.append('TransactionDT')

# Đảm bảo các cột thiết yếu có trong df_processed trước khi gộp
final_columns = list(set(top_n_features + essential_cols))
df_final = df_processed[final_columns].copy()

# Sắp xếp lại cột: Cột thiết yếu trước, đặc trưng sau
other_cols = [col for col in final_columns if col not in essential_cols]
df_final = df_final[essential_cols + other_cols]

# Lưu file trung gian (chưa scale)
lstm_ready_path = os.path.join(OUTPUT_DIR_STEP1, "train_data_LSTM_ready.csv")
df_final.to_csv(lstm_ready_path, index=False)
print(f" -> Đã lưu dữ liệu tinh gọn (chưa scale) vào: {lstm_ready_path}")

# =========================================
# [BỔ SUNG TẠI ĐÂY] TÍNH VÀ LƯU TRAIN STATS
# =========================================
print("\n📊 Đang tính toán và lưu thống kê (train_stats)...")
train_stats = {}
# Tính trên df_final vì nó chứa dữ liệu gốc chưa scale của các feature đã chọn
if 'card1' in df.columns and 'TransactionAmt' in df.columns:
     # Lưu ý: Cần lấy từ df gốc ban đầu hoặc đảm bảo df_final có đủ cột
     # Tốt nhất là dùng df_processed vì nó chắc chắn có đủ cột trước khi bị cắt giảm
    train_stats['mean'] = df_processed.groupby('card1')['TransactionAmt'].mean().to_dict()
    train_stats['std'] = df_processed.groupby('card1')['TransactionAmt'].std().fillna(1.0).to_dict()
    
    stats_path = os.path.join(OUTPUT_DIR_FINAL, 'train_stats.joblib')
    joblib.dump(train_stats, stats_path)
    print(f"✅ Đã lưu train_stats vào: {stats_path}")
else:
    print("⚠️ Cảnh báo: Không tìm thấy 'card1' hoặc 'TransactionAmt' để tính stats.")
# 4.2. Chuẩn hóa (MinMax Scaling) cho các đặc trưng
print(" -> Đang chuẩn hóa dữ liệu về [0, 1]...")
scaler = MinMaxScaler()
# Chỉ scale các cột đặc trưng (không scale ID, nhãn, thời gian)
df_final[top_n_features] = df_final[top_n_features].replace([np.inf, -np.inf], np.nan).fillna(0)
df_final[top_n_features] = scaler.fit_transform(df_final[top_n_features])


⏳ Đang đọc dữ liệu từ ../data_clear2/train_split.csv...
✅ Đã đọc dữ liệu. Shape: (472432, 176)

🔄 [BƯỚC 1] Xử lý sơ bộ (Encoding & NaN)...
 -> Tìm thấy 0 cột categorical cần mã hóa.
 -> Hoàn tất Label Encoding.

🔍 [BƯỚC 2] Lọc đặc trưng (Feature Selection)...
 -> Đang lọc đặc trưng ít biến thiên (Variance < 0.05)...
    => Còn lại 142 đặc trưng.
 -> Đang lọc đặc trưng tương quan cao (> 0.99)...
    => Đã loại bỏ 13 đặc trưng dư thừa. Còn lại 129 đặc trưng.
 -> Đang chạy RandomForest để xếp hạng độ quan trọng...
    => Đã chọn ra 100 đặc trưng quan trọng nhất.

⚖️ [BƯỚC 3] Tạo dataset cuối cùng và Chuẩn hóa (Scaling)...
 -> Đã lưu dữ liệu tinh gọn (chưa scale) vào: processed_data\train_data_LSTM_ready.csv

📊 Đang tính toán và lưu thống kê (train_stats)...
✅ Đã lưu train_stats vào: processed_data_final_v1\train_stats.joblib
 -> Đang chuẩn hóa dữ liệu về [0, 1]...


In [12]:
# =========================================
# 5. LƯU KẾT QUẢ CUỐI CÙNG
# =========================================
print("\n💾 [BƯỚC 4] Lưu trữ kết quả cuối cùng...")

# Lưu dataset đã scale
final_output_csv = os.path.join(OUTPUT_DIR_FINAL, "train_data_optimized.csv")
df_final.to_csv(final_output_csv, index=False)
print(f"✅ Đã lưu dữ liệu hoàn chỉnh vào: {final_output_csv}")

# In ra cấu trúc cuối cùng của DataFrame
print("\nCấu trúc cuối cùng của DataFrame:")
print(df_final.info())

# Lưu các file phụ trợ quan trọng
joblib.dump(top_n_features, os.path.join(OUTPUT_DIR_FINAL, "final_feature_list.joblib"))
joblib.dump(scaler, os.path.join(OUTPUT_DIR_FINAL, "scaler_v1.joblib"))
joblib.dump(label_encoders, os.path.join(OUTPUT_DIR_FINAL, "label_encoders.joblib")) 
print("✅ Đã lưu danh sách đặc trưng, scaler và encoder.")

print("\n🎉 QUY TRÌNH HOÀN TẤT! Dữ liệu đã sẵn sàng cho việc tạo chuỗi LSTM.")
gc.collect()


💾 [BƯỚC 4] Lưu trữ kết quả cuối cùng...
✅ Đã lưu dữ liệu hoàn chỉnh vào: processed_data_final_v1\train_data_optimized.csv

Cấu trúc cuối cùng của DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472432 entries, 0 to 472431
Columns: 102 entries, TransactionID to M9_T
dtypes: float64(100), int64(2)
memory usage: 367.6 MB
None
✅ Đã lưu danh sách đặc trưng, scaler và encoder.

🎉 QUY TRÌNH HOÀN TẤT! Dữ liệu đã sẵn sàng cho việc tạo chuỗi LSTM.


0

In [13]:
# =========================================
# 6. ĐÁNH GIÁ NHANH CHẤT LƯỢNG DỮ LIỆU (BỔ SUNG)
# =========================================
print("\n🚀 [BƯỚC 5] Đánh giá nhanh chất lượng dữ liệu bằng Random Forest...")
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 6.1. Chuẩn bị dữ liệu cho đánh giá
# Lấy các đặc trưng đã chọn và đã scale từ df_final
X_eval = df_final[top_n_features]
y_eval = df_final['isFraud']

# Chia train/test (ví dụ 80/20) để đánh giá khách quan
X_train_eval, X_test_eval, y_train_eval, y_test_eval = train_test_split(
    X_eval, y_eval, test_size=0.2, random_state=42, stratify=y_eval
)

# 6.2. Huấn luyện Random Forest đơn giản
print(" -> Đang huấn luyện mô hình đánh giá...")
rf_eval = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
rf_eval.fit(X_train_eval, y_train_eval)

# 6.3. Dự đoán và tính toán F1-score
print(" -> Đang tính toán các chỉ số đánh giá...")
y_pred_eval = rf_eval.predict(X_test_eval)

f1 = f1_score(y_test_eval, y_pred_eval)
conf_matrix = confusion_matrix(y_test_eval, y_pred_eval)
report = classification_report(y_test_eval, y_pred_eval, digits=4)

print("\n📊 --- KẾT QUẢ ĐÁNH GIÁ CHẤT LƯỢNG DỮ LIỆU ---")
print(f"✅ F1-Score (lớp Fraud): {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

# Đánh giá sơ bộ dựa trên F1
if f1 > 0.7:
    print("\n🌟 Đánh giá: Dữ liệu có chất lượng RẤT TỐT để bắt đầu mô hình hóa.")
elif f1 > 0.5:
    print("\n👍 Đánh giá: Dữ liệu có chất lượng KHÁ TỐT.")
else:
    print("\n⚠️ Đánh giá: F1-score còn thấp. Có thể cần xem lại Feature Engineering hoặc cân bằng dữ liệu.")


🚀 [BƯỚC 5] Đánh giá nhanh chất lượng dữ liệu bằng Random Forest...
 -> Đang huấn luyện mô hình đánh giá...
 -> Đang tính toán các chỉ số đánh giá...

📊 --- KẾT QUẢ ĐÁNH GIÁ CHẤT LƯỢNG DỮ LIỆU ---
✅ F1-Score (lớp Fraud): 0.6347

Confusion Matrix:
[[91101    66]
 [ 1746  1574]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9812    0.9993    0.9902     91167
           1     0.9598    0.4741    0.6347      3320

    accuracy                         0.9808     94487
   macro avg     0.9705    0.7367    0.8124     94487
weighted avg     0.9804    0.9808    0.9777     94487


👍 Đánh giá: Dữ liệu có chất lượng KHÁ TỐT.
